# Baseline

In [1]:
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int,
        out_dim: int,
        num_classes: int
    ):
        super().__init__()
        self.conv1 = nn.Conv2d(in_dim, hidden_dim, 5)
        self.act1 = nn.ReLU()
        self.conv2 = nn.Conv2d(hidden_dim, out_dim, 5)
        self.act2 = nn.Softmax(dim=1)
                
        self._classification_head = nn.Sequential(
            nn.Linear(out_dim, out_dim),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(out_dim, num_classes),
            nn.LogSoftmax(dim=1),
        )

    def forward(self, x):
        z = self.act1(self.conv1(x))
        z = self.act2(self.conv2(z))
        z = self._classification_heads(z)
        return z

In [4]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

NameError: name 'trainloader' is not defined